# Binary knapsack
- N items:
    + weight[1:N]
    + value[1:N]

- Find the maximum value of combined items that not exceeded W
- Constraints
    + $N \le 10^4$
    + $W \le 10^6$


```C++
int N, W;
int wei[10003];
int val[10003];
```

#### Example
```
<!--  Input -->
4 5
1 2 4 5
5 4 8 6

<!--  Output -->
13
```


# Solution

<img src="./img/10.jpg" alt="drawing" width="800"/>



## 1. BottomUp Recursive
- **Notes**
    + Easy to implement, natural approach
    + Not optimized in runtime (mostly complete search)
    + Try applying pruning to reduce time complexity
    + **Recursive = DFS**

#### 1.1 CompleteSearch - Time TLE - Space O(NW)

```C++
// state: val = (i, w)
int res;
int dp[10003][1000003];
void get(int i, int w) {
    if(i == N) {
        res = max(res, dp[i][w]);
        return;
    }
    if(dp[i][w] == -1) return;

    int cur = dp[i][w];

    // Choose i+1
    if(w+wei[i+1] <= W) {
        dp[i+1][w+wei[i+1]] = max(dp[i+1][w+wei[i+1]], cur + val[i+1]);
        get(i+1, w+wei[i+1]);
    }

    // Not choose i+1
    dp[i+1][w] = max(dp[i+1][w], cur);
    get(i+1, w);
}

int solveDP() {
    // Init
    memset(dp, -1, sizeof(dp));
    dp[0][0] = 0;
    res = -1;

    // DP
    get(0,0);
    
    // Get results
    return res;
}
```

#### 1.2 Optimize w space - Time TLE - Space O(NW)

```C++
// state: val = (i, w)
unordered_map<int, int> dp[10003];
int res;
void get(int i, int w) {
    if(i == N) {
        res = max(res, dp[i][w]);
        return;
    }
    if(dp[i].count(w) == NULL) return;
    int cur = dp[i][w];

    // Choose i+1
    if(w+wei[i+1] <= W) {
        dp[i+1][w+wei[i+1]] = max(dp[i+1][w+wei[i+1]], cur + val[i+1]);
        get(i+1, w+wei[i+1]);
    }

    // Not choose i+1
    dp[i+1][w] = max(dp[i+1][w], cur);
    get(i+1, w);
}

int solveDP() {
    // Init
    dp[0][0] = 0;
    res = -1;

    // DP
    get(0,0);

    // Get results
    return res;
}
```

#### 1.3 Optimize i, w space  - Time TLE - Space O(W)
- **Note**: `dp[i+1]` only depend on `dp[i]`

```C++
int res;
void get(int i, int w, unordered_map<int, int> dp_) {
    if(i == N) {
        res = max(res, dp_[w]);
        return;
    }
    if(dp_.count(w) == NULL) return;

    unordered_map<int,int> dp;

    // Choose i+1
    if(w+wei[i+1] <= W) {
        dp[w+wei[i+1]] = dp_[w] + val[i+1];
        get(i+1, w+wei[i+1], dp);
    }

    // Not choose i+1
    dp[w] = dp_[w];
    get(i+1, w, dp);
}

int solveDP() {
    // DP
    res = -1;
    get(0,0, unordered_map<int,int>{{0,0}});

    // Get results
    return res;
}
```

#### 1.4 Pruning Optimization - (Backtracking + DP) - Time less TLE - Space O(NW)

```C++
// state: val = (i, w)
int dp[10003][1000003];
int res;
void get(int i, int w) {
    int cur = dp[i][w];

    // Get results
    if(i == N) {
        res = max(res, cur);
        return;
    }
    if(dp[i][w] == -1) return;

    // Choose i+1
    if(w+wei[i+1] <= W &&
            dp[i+1][w+wei[i+1]] < cur + val[i+1]) {
        dp[i+1][w+wei[i+1]] = cur + val[i+1];
        get(i+1, w+wei[i+1]);
    }

    // Not choose i+1
    if(i+1 <= N &&
            dp[i+1][w] < cur) {
        dp[i+1][w] = cur;
        get(i+1, w);
    }
}

int solveDP() {
    // Init
    memset(dp,-1, sizeof(dp));
    res = -1;

    // dfs - DP
    dp[0][0] = 0;
    get(0,0);
    return res;
}
```


## 2. Topdown Recursive - Time O(N*W) - Space O(N*W)
- **Notes**:
    + Topdown dfs optimized in time complexity than bottom-up dfs
    + Implementation complexity(easy -> hard): bottom up dfs < topdown dfs < loop 
    + Optimization complexity(slow -> fast): bottom up dfs < topdown dfs < loop
    + **Recursive = DFS**

```C++
// state: val = (i, w)
int dp[10003][1000003];
int get(int i, int w) {
    if(i==0 && w>=0) return 0;
    if(i<0 || w<0) return INT_MIN;
    if(dp[i][w] != -1) return dp[i][w];

    // Choose i
    dp[i][w] = max(dp[i][w], get(i-1, w-wei[i]) + val[i]);

    // Not choose i
    dp[i][w] = max(dp[i][w], get(i-1, w));

    return dp[i][w];
}
int solveDP() {
    memset(dp, -1, sizeof(dp));
    return get(N,W);
}
```


## 3. Loop
- **Notes**:
    + Hardest implementation
    + Most optimal
    + **Loop = BFS**

#### Loop - Time O(N*W) - Space O(N*W)

```C++
// state: val = (i, w)
int dp[10003][1000003];
int solveDP() {
    // Init
    memset(dp, -1, sizeof(dp));
    dp[0][0] = 0;

    // DP
    for(int i=0; i<N; ++i) for(int w=0; w<=W; ++w) {
        int cur = dp[i][w];
        if(cur == -1) continue;

        // Choose i+1
        if(w+wei[i+1] <= W)
            dp[i+1][w+wei[i+1]] = max(dp[i+1][w+wei[i+1]], cur + val[i+1]);

        // Not choose i+1
        dp[i+1][w] = max(dp[i+1][w], cur);
    }

    // Get results
    int res = -1;
    for(int w=0; w<=W; ++w) res = max(res, dp[N][w]);
    return res;
}
```

#### Loop with optimized i - Time O(N*W) - Space O(W)
- **Note**: `dp[i+1]` only depend on `dp[i]`

```C++
int dp[2][100003];
int solveDP() {
    // Init
    memset(dp, -1, sizeof(dp));
    dp[0][0] = 0;
    dp[1][0] = 0;

    // DP
    int cur, nex;
    for(int i=0; i<N; ++i) for(int w=0; w<=W; ++w) {
        if(i%2 == 0) {
            cur = 0;
            nex = 1;
        } else {
            cur = 1;
            nex = 0;
        }
        if(dp[cur][w] == -1) continue;

        // Choose i+1
        if(w+wei[i+1] <= W)
            dp[nex][w+wei[i+1]] = max(dp[nex][w+wei[i+1]], dp[cur][w] + val[i+1]);

        // Not choose i+1
        dp[nex][w] = max(dp[nex][w], dp[cur][w]);
    }

    // Get results
    int res = -1;
    for(int idx=0; idx<2; ++idx) for(int w=0; w<=W; ++w) {
        res = max(res, dp[idx][w]);
    }
    return res;
}
```